In [1]:
import bokeh
import shutil
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import defaultdict, namedtuple
from pathlib import Path
from queue import Queue
from time import sleep
from dateutil import parser
import tempfile
from watchdog.observers import Observer, ObserverType
from bokeh.plotting import figure
from bokeh.io import show
from bokeh.io import push_notebook



from ogstools.logparser.log_file_handler import (
    LogFileHandler,
    normalize_regex,
    parse_line,
    select_regex,
)
from ogstools.logparser.regexes import (
    Context,
    StepStatus,
    Termination,
    TimeStepEnd,
    TimeStepStart,
)   
print("Bokeh Version : {}".format(bokeh.__version__))

Bokeh Version : 3.7.3


In [2]:
import requests
import time
from datetime import datetime
from bokeh.io import output_notebook
from bokeh.io import push_notebook

output_notebook()

Loading BokehJS ...

In [3]:
from bokeh.models import ColumnDataSource

data_source = ColumnDataSource(data = {"time_step": [], "step_size": []})

In [4]:
chunk_size=4096
delay = 0.0

original_file = Path("out_test_transition-sand.log")
print(str(original_file.parent))
#temp_dir = Path(
#            tempfile.mkdtemp(
#                f"test_v2_coupled_with_producer_{chunk_size}_{delay}"
#            )
#        )

#temp_dir.mkdir(parents=True, exist_ok=True)

#new_file = temp_dir / "ht.log"
records: Queue = Queue()
observer: ObserverType = Observer()
status: Context = Context()
#shutil.rmtree(new_file, ignore_errors=True)
handler = LogFileHandler(
    original_file,
    queue=records,
    status=status,
    stop_callback=lambda: (print("Stop Observer"), observer.stop()),
)

observer.schedule(handler, path=str(original_file.parent), recursive=False)

print("Starting observer...")

observer.start()

.
Starting observer...


In [5]:
fig = figure(  width=900, height=450,
             tooltips=[("step_size", "@step_size"),],
             title="time"
            )

fig.line(x="time_step", y="step_size", line_color="tomato", line_width=3.0, source=data_source)

fig.xaxis.axis_label = "time_step (s)"
fig.yaxis.axis_label = "start_time (s)"

handle_line_chart = show(fig, notebook_handle=True)

In [6]:
while True:
    item = records.get()
    if isinstance(item, Termination):
        print(f"Consumer: Termination signal ({item}) received. Exiting.")
        break
    #print(item)
    if isinstance(item,TimeStepStart):
        print(f"Timestep: {item.time_step}")
        new_row = {"step_size": [item.step_size,], "time_step": [item.time_step,]}
        data_source.stream(new_row)
        push_notebook(handle=handle_line_chart)
    
    #time.sleep(0.1)
observer.join()

KeyboardInterrupt: 

In [ ]:
item